In [251]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import mutual_info_classif
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder, Normalizer


In [96]:
df_public = pd.read_csv('survey_results_public.csv')

In [5]:
df_schema = pd.read_csv('survey_results_schema.csv')

In [12]:
df_public.shape

(51392, 154)

In [78]:
df_public.head()

,Respondent,Professional,ProgramHobby,Country,University,EmploymentStatus,FormalEducation,MajorUndergrad,HomeRemote,CompanySize,...,StackOverflowMakeMoney,Gender,HighestEducationParents,Race,SurveyLong,QuestionsInteresting,QuestionsConfusing,InterestedAnswers,Salary,ExpectedSalary
0,1,Student,"Yes, both",United States,No,"Not employed, and not looking for work",Secondary school,NaN,NaN,NaN,...,Strongly disagree,Male,High school,White or of European descent,Strongly disagree,Strongly agree,Disagree,Strongly agree,NaN,NaN
1,2,Student,"Yes, both",United Kingdom,"Yes, full-time",Employed part-time,Some college/university study without earning ...,Computer science or software engineering,"More than half, but not all, the time",20 to 99 employees,...,Strongly disagree,Male,A master's degree,White or of European descent,Somewhat agree,Somewhat agree,Disagree,Strongly agree,NaN,37500.0
2,3,Professional developer,"Yes, both",United Kingdom,No,Employed full-time,Bachelor's degree,Computer science or software engineering,"Less than half the time, but at least one day ...","10,000 or more employees",...,Disagree,Male,A professional degree,White or of European descent,Somewhat agree,Agree,Disagree,Agree,113750.0,NaN
3,4,Professional non-developer who sometimes write...,"Yes, both",United States,No,Employed full-time,Doctoral degree,A non-computer-focused engineering discipline,"Less than half the time, but at least one day ...","10,000 or more employees",...,Disagree,Male,A doctoral degree,White or of European descent,Agree,Agree,Somewhat agree,Strongly agree,NaN,NaN
4,5,Professional developer,"Yes, I program as a hobby",Switzerland,No,Employed full-time,Master's degree,Computer science or software engineering,Never,10 to 19 employees,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [156]:
df_schema[df_schema.Column == 'ImportantBenefits']['Question'].values

array(['When it comes to compensation and benefits, other than base salary, which of the following are most important to you?'],
      dtype=object)

In [103]:
df_proDevel = df_public[df_public.Professional == 'Professional developer']

In [177]:
top_ratio_dict = {}
# Features' samples number less than sample_size in the data set will be dropped.
sample_size = 1000
min_tgi = 1.2
top_ratio_num = 3

for column in df_public.columns:
    
    keep_len = (df_public[column].value_counts() > sample_size).sum()   
    #? 都用keep_len是否准确？
    top_five_ratio = ((df_proDevel[column].value_counts().iloc[:keep_len] / df_proDevel[column].shape[0])  \
                               /(df_public[column].value_counts().iloc[:keep_len] / df_public[column].shape[0]))   \
                               .sort_values(ascending=False).iloc[:top_num]
    # Skip the column being empty or all values being NAN.
    if np.isnan(top_five_ratio.values).sum() == len(top_five_ratio.values):
        continue
        
    filter_condition =  top_five_ratio.values[:1][0] > min_tgi # or np.isnan(top_five_ratio.values).any() 
                        
    if filter_condition:
        top_ratio_dict[column] = top_five_ratio
         

In [225]:
columns = []
for column,top_ratio in top_ratio_dict.items():
    columns.append(column)
    #print(top_ratio)
#columns

In [183]:
df_public.head()

,Respondent,Professional,ProgramHobby,Country,University,EmploymentStatus,FormalEducation,MajorUndergrad,HomeRemote,CompanySize,...,StackOverflowMakeMoney,Gender,HighestEducationParents,Race,SurveyLong,QuestionsInteresting,QuestionsConfusing,InterestedAnswers,Salary,ExpectedSalary
0,1,Student,"Yes, both",United States,No,"Not employed, and not looking for work",Secondary school,NaN,NaN,NaN,...,Strongly disagree,Male,High school,White or of European descent,Strongly disagree,Strongly agree,Disagree,Strongly agree,NaN,NaN
1,2,Student,"Yes, both",United Kingdom,"Yes, full-time",Employed part-time,Some college/university study without earning ...,Computer science or software engineering,"More than half, but not all, the time",20 to 99 employees,...,Strongly disagree,Male,A master's degree,White or of European descent,Somewhat agree,Somewhat agree,Disagree,Strongly agree,NaN,37500.0
2,3,Professional developer,"Yes, both",United Kingdom,No,Employed full-time,Bachelor's degree,Computer science or software engineering,"Less than half the time, but at least one day ...","10,000 or more employees",...,Disagree,Male,A professional degree,White or of European descent,Somewhat agree,Agree,Disagree,Agree,113750.0,NaN
3,4,Professional non-developer who sometimes write...,"Yes, both",United States,No,Employed full-time,Doctoral degree,A non-computer-focused engineering discipline,"Less than half the time, but at least one day ...","10,000 or more employees",...,Disagree,Male,A doctoral degree,White or of European descent,Agree,Agree,Somewhat agree,Strongly agree,NaN,NaN
4,5,Professional developer,"Yes, I program as a hobby",Switzerland,No,Employed full-time,Master's degree,Computer science or software engineering,Never,10 to 19 employees,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [196]:
JobSatisfaction 

Respondent               True
Professional             True
ProgramHobby             True
Country                  True
University               True
                        ...  
QuestionsInteresting    False
QuestionsConfusing      False
InterestedAnswers       False
Salary                  False
ExpectedSalary          False
Length: 154, dtype: bool

In [222]:
numbernic = ['CareerSatisfaction','JobSatisfaction','HoursPerWeek','StackOverflowSatisfaction']

In [ ]:
df_public.columns

In [202]:
transfer_df = df_public[df_public.dtypes[df_public.dtypes =='object'].index]

In [267]:
(transfer_df.isnull().sum()==0).values

array([ True,  True,  True,  True,  True,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [223]:
numbernic_df = df_public[numbernic]

In [243]:
ct = ColumnTransformer([('LabelEncoder', LabelEncoder(), [1])], remainder='passthrough')


In [268]:
preprocessor = make_column_transformer((OrdinalEncoder(),[0, 146]),remainder="passthrough",sparse_threshold=0.3)

In [269]:
preprocessor.fit_transform(transfer_df.values)

ValueError: Input contains NaN

In [244]:
ct.fit_transform(transfer_df.values)

TypeError: fit_transform() takes 2 positional arguments but 3 were given